In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 40.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
from collections import defaultdict # defaultdict는 키가 없을 경우 자동으로 기본값을 생성
from konlpy.tag import Okt

# 치킨 리뷰 데이터 생성
chicken_reviews = {
    'document': [
        "치킨이 너무 바삭바삭하고 맛있어요",
        "양이 많고 배달도 빨라서 좋았습니다",
        "치킨이 식어서 왔어요 실망이에요",
        "양념이 너무 달고 짜요 별로에요",
        "고소하고 담백해서 잘 먹었습니다",
        "치킨이 너무 기름져요 별로였어요",
        "순살이라 먹기 편하고 맛있었어요",
        "가격대비 양이 적어요 아쉽네요",
        "소스가 끝내주게 맛있어요 추천합니다",
        "배달 포장이 엉망이에요 실망입니다",
        "치킨 크기가 실하고 맛있어요",
        "너무 짜고 매워서 못 먹겠어요",
        "콜라랑 같이 먹으니 완벽해요",
        "치킨이 차갑게 와서 맛이 없었어요",
        "바삭하고 육즙이 살아있어요 최고",
        "후라이드 치킨의 황금비율을 찾은 것 같아요",
        "배달시간이 너무 오래 걸려서 실망했어요",
        "치킨무가 신선하고 아삭아삭해요",
        "양념치킨인데 너무 달기만 하고 맵지가 않아요",
        "가성비 최고의 치킨집이에요",
        "치킨이 너무 작아서 속상해요",
        "리뷰 보고 주문했는데 기대 이상이에요",
        "오늘 치킨이 덜 익어서 왔어요",
        "단골집인데 오늘도 역시 맛있네요",
        "치킨이 너무 짜서 물을 많이 마셨어요",
        "신메뉴인데 완전 신선하고 맛있어요",
        "치킨 뼈가 너무 작아서 먹기 불편해요",
        "소스가 따로 포장되어 있어서 좋았어요",
        "치킨이 너무 질겨서 먹기 힘들었어요",
        "서비스로 주신 음료수가 맛있네요"
    ],
    'label': [
        1,  # 긍정
        1,
        0,  # 부정
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1,
        0,
        1
    ]
}

# DataFrame 생성
data = pd.DataFrame(chicken_reviews)

# 훈련 데이터와 테스트 데이터로 분리
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 데이터 전처리 (특수문자 제거, 결측값 제거)
def clean_text(text):
    text = re.sub(r'[^가-힣0-9\s]', '', text)  # 한글, 숫자, 공백을 제외한 모든 문자 제거
    return text

train_data['document'] = train_data['document'].apply(lambda x: clean_text(str(x)))
test_data['document'] = test_data['document'].apply(lambda x: clean_text(str(x)))

# 결측값 제거
train_data = train_data.dropna(subset=['document'])
test_data = test_data.dropna(subset=['document'])

# N-gram 생성을 위한 함수
def generate_ngram(text, n):
    words = text.split()  # 공백을 기준으로 단어 분리
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i + n])
        ngrams.append(ngram)
    return ngrams

# N-gram을 딕셔너리에 저장하는 함수
def build_ngram_dict(corpus, n):
    ngram_dict = defaultdict(int)
    for sentence in corpus:
        ngrams = generate_ngram(sentence, n)
        for ngram in ngrams:
            ngram_dict[ngram] += 1
    return ngram_dict

# N-gram 벡터화 함수 (문장을 N-gram 벡터로 변환)
def ngram_vectorize(corpus, ngram_dict):
    vectors = []
    for sentence in corpus:
        ngram_vector = [0] * len(ngram_dict)  # N-gram 수만큼 0으로 채운 리스트
        ngrams = generate_ngram(sentence, 2)  # 2-gram 생성
        for ngram in ngrams:
            if ngram in ngram_dict:
                idx = list(ngram_dict.keys()).index(ngram)  # N-gram의 인덱스를 찾음
                ngram_vector[idx] += 1
        vectors.append(ngram_vector)
    return vectors


In [4]:
# N-gram 사전 구축 (1-gram + 2-gram 사용)
ngram_dict = build_ngram_dict(train_data['document'], 2)

# 훈련 데이터와 테스트 데이터를 N-gram 벡터로 변환
X_train = ngram_vectorize(train_data['document'], ngram_dict)
X_test = ngram_vectorize(test_data['document'], ngram_dict)

# 라벨 (긍정 = 1, 부정 = 0)
y_train = train_data['label']
y_test = test_data['label']

# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8333

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.83      1.00      0.91         5

    accuracy                           0.83         6
   macro avg       0.42      0.50      0.45         6
weighted avg       0.69      0.83      0.76         6



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
ngram_dict

defaultdict(int,
            {'치킨이 너무': 5,
             '너무 질겨서': 1,
             '질겨서 먹기': 1,
             '먹기 힘들었어요': 1,
             '너무 짜서': 1,
             '짜서 물을': 1,
             '물을 많이': 1,
             '많이 마셨어요': 1,
             '콜라랑 같이': 1,
             '같이 먹으니': 1,
             '먹으니 완벽해요': 1,
             '너무 바삭바삭하고': 1,
             '바삭바삭하고 맛있어요': 1,
             '고소하고 담백해서': 1,
             '담백해서 잘': 1,
             '잘 먹었습니다': 1,
             '배달시간이 너무': 1,
             '너무 오래': 1,
             '오래 걸려서': 1,
             '걸려서 실망했어요': 1,
             '너무 기름져요': 1,
             '기름져요 별로였어요': 1,
             '치킨이 차갑게': 1,
             '차갑게 와서': 1,
             '와서 맛이': 1,
             '맛이 없었어요': 1,
             '너무 짜고': 1,
             '짜고 매워서': 1,
             '매워서 못': 1,
             '못 먹겠어요': 1,
             '오늘 치킨이': 1,
             '치킨이 덜': 1,
             '덜 익어서': 1,
             '익어서 왔어요': 1,
             '양이 많고': 1,
             '많고 배달도': 1,
             '배달도 빨라서': 1,


In [6]:
X_train

[[1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0